In [1]:
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

In [2]:
import tensorflow_hub as hub
embed = hub.load("../../model/")

data = pd.read_csv("../../data/english_dataset/english_dataset.tsv",sep="\t")
data.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE,NONE
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,NOT,NONE,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,NOT,NONE,NONE
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,NOT,NONE,NONE


In [3]:
def create_label(labels, data):
    return data.map(labels, na_action='ignore')

In [9]:
def create_model(n_class):
    print("CLASSES:", n_class)
    model = Sequential()
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(n_class, activation='softmax'))
    return model

In [10]:
def train(model, X_train, X_test, y_train, y_test):
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=5, batch_size=32)
    print(model.evaluate(X_test, y_test))
    

In [12]:
# label_task1, label_task2, label_task3 = {'NOT': 0, 'HOF': 1}, {'HATE':0, 'OFFN':1, 'PRFN':2}, {'TIN':0, 'UNT':1}
label = [{'NOT': 0, 'HOF': 1}, {'HATE':0, 'OFFN':1, 'PRFN':2, 'NONE':3}, {'TIN':0, 'UNT':1, 'NONE':2}]

In [13]:
for i in range(1,4):
    col = "task_"+str(i)
    y = create_label(label[i-1], data[col])
    x = data[col]
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
    
    X_train_embed = embed(X_train)
    X_test_embed = embed(X_test)
    
    model = create_model(n_class = len(label[i-1]))
    train(model, X_train_embed, X_test_embed, y_train, y_test)

CLASSES: 2
Epoch 1/5
123/123 [==============================] - 1s 3ms/step - loss: 0.1278 - accuracy: 0.9660
Epoch 2/5
123/123 [==============================] - 0s 3ms/step - loss: 4.8743e-05 - accuracy: 1.0000
Epoch 3/5
123/123 [==============================] - 0s 2ms/step - loss: 3.1953e-05 - accuracy: 1.0000
Epoch 4/5
123/123 [==============================] - 0s 2ms/step - loss: 2.1449e-05 - accuracy: 1.0000
Epoch 5/5
61/61 [==============================] - 0s 1ms/step - loss: 1.1637e-05 - accuracy: 1.0000
[1.1636541785264853e-05, 1.0]
CLASSES: 4
Epoch 1/5
123/123 [==============================] - 1s 3ms/step - loss: 0.3077 - accuracy: 0.9500
Epoch 2/5
123/123 [==============================] - 0s 2ms/step - loss: 1.5050e-04 - accuracy: 1.0000
Epoch 3/5
123/123 [==============================] - 0s 2ms/step - loss: 5.8758e-05 - accuracy: 1.0000
Epoch 4/5
123/123 [==============================] - 0s 2ms/step - loss: 2.7110e-05 - accuracy: 1.0000
Epoch 5/5
61/61 [==============

In [ ]:
'''
labels = {'TIN':0, 'UNT':1, 'NONE':3}
y = data['task_3'].map(labels, na_action='ignore')
x = data['text']
'''